In [2]:
import sys
import os
import gc
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np

path_build = "./build/"
path_embed = "./embed/"
path_list = "./feature_series/"
path_mnt = "./mnt/"
path_sub = "./sub/"


pd.set_option('display.max_columns', None)

In [3]:
click_log = pd.read_pickle(path_build + 'click_log.pkl')
train_user = pd.read_pickle(path_build + 'train_user.pkl')
print(click_log.shape, train_user.shape)

(133878445, 9) (3000000, 3)


In [4]:
click_log.sort_values(by=['user_id','time'], ascending=[True,True], inplace=True)
click_log = click_log.reset_index(drop=True)
click_log = click_log[click_log['click_times']<=8] #去除点击次数99.99%外的数据记录
print(click_log.shape)
click_log.head()

(133869604, 9)


,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,1,821396,1,724607,10005,5,7293,326
1,20,1,209778,1,188507,136,2,9702,6
2,20,1,877468,1,773445,10005,5,29455,106
3,39,1,1683713,1,1458878,10005,5,14668,326
4,40,1,122032,1,109959,1334,2,11411,100


In [5]:
# click_log['creative_id_times'] = click_log['creative_id'].astype(np.int64)*10000000 + click_log['click_times']
# click_log['ad_id_times'] = click_log['ad_id'].astype(np.int64)*10000000 + click_log['click_times']
# click_log['product_id_times'] = click_log['product_id'].astype(np.int64)*10000000 + click_log['click_times']
# click_log['advertiser_id_times'] = click_log['advertiser_id'].astype(np.int64)*10000000 + click_log['click_times']
# click_log['product_category_times'] = click_log['product_category'].astype(np.int64)*10000000 + click_log['click_times']
# click_log['industry_times'] = click_log['industry'].astype(np.int64)*10000000 + click_log['click_times']


# click_log['creative_id_times'] = click_log['creative_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
# click_log['ad_id_times'] = click_log['ad_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
# click_log['product_id_times'] = click_log['product_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
# click_log['advertiser_id_times'] = click_log['advertiser_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
# click_log['product_category_times'] = click_log['product_category'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
# click_log['industry_times'] = click_log['industry'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')

click_log['product_advertiser'] = click_log['product_id'].astype(np.int64)*10000000 + click_log['advertiser_id']
click_log['product_industry'] = click_log['product_id'].astype(np.int64)*10000000 + click_log['industry']
click_log['advertiser_industry'] = click_log['advertiser_id'].astype(np.int64)*10000000 + click_log['industry']

click_log.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,product_advertiser,product_industry,advertiser_industry
0,20,1,821396,1,724607,10005,5,7293,326,100050007293,100050000326,72930000326
1,20,1,209778,1,188507,136,2,9702,6,1360009702,1360000006,97020000006
2,20,1,877468,1,773445,10005,5,29455,106,100050029455,100050000106,294550000106
3,39,1,1683713,1,1458878,10005,5,14668,326,100050014668,100050000326,146680000326
4,40,1,122032,1,109959,1334,2,11411,100,13340011411,13340000100,114110000100


In [6]:
click_log = multi_column_LabelEncoder(click_log, 
                                      columns=['product_advertiser','product_industry','advertiser_industry',
                                              ])

print(click_log['product_advertiser'].nunique())
print(click_log['product_industry'].nunique())
print(click_log['advertiser_industry'].nunique())
# print(click_log['product_category'].nunique())
# print(click_log['advertiser_id'].nunique())
# print(click_log['industry'].nunique())
# print(click_log['creative_id_times'].nunique())
# print(click_log['ad_id_times'].nunique())
# print(click_log['product_id_times'].nunique())
# print(click_log['advertiser_id_times'].nunique())
# print(click_log['product_category_times'].nunique())
# print(click_log['industry_times'].nunique())

product_advertiser LabelEncoder......
product_industry LabelEncoder......
advertiser_industry LabelEncoder......
LabelEncoder Successfully!
87851
53933
85835


In [10]:
from collections import defaultdict
import sys
# creative_id = defaultdict(list)
# ad_id = defaultdict(list)
# product_id = defaultdict(list)
# product_category = defaultdict(list)
# advertiser_id = defaultdict(list)
# industry = defaultdict(list)
# click_times = defaultdict(list)
# time = defaultdict(list)
# creative_id_times = defaultdict(list)
# ad_id_times = defaultdict(list)
# product_id_times = defaultdict(list)
# advertiser_id_times = defaultdict(list)
# product_category_times = defaultdict(list)
# industry_times = defaultdict(list)

product_advertiser = defaultdict(list)
product_industry = defaultdict(list)
advertiser_industry = defaultdict(list)


for user,f1,f2,f3  in zip(
                             click_log['user_id'], 
#                             click_log['creative_id'],
#                             click_log['ad_id'],
#                             click_log['product_id'],
#                             click_log['product_category'],
#                             click_log['advertiser_id'],
#                             click_log['industry'],
#                             click_log['click_times'],
#                             click_log['time'],
#                             click_log['creative_id_times'],
#                             click_log['ad_id_times'],
#                             click_log['product_id_times'],
#                             click_log['advertiser_id_times'],
#                             click_log['product_category_times'],
#                             click_log['industry_times'],
                            click_log['product_advertiser'], 
                            click_log['product_industry'],
                            click_log['advertiser_industry']
):
#     creative_id[user].append(f1)
#     ad_id[user].append(f2)
#     product_id[user].append(f3)
#     product_category[user].append(f4)
#     advertiser_id[user].append(f5)
#     industry[user].append(f6)
#     click_times[user].append(f7)
#     time[user].append(f8)
    
#     creative_id_times[user].append(f9)
#     ad_id_times[user].append(f10)
#     product_id_times[user].append(f11)
#     advertiser_id_times[user].append(f12)
#     product_category_times[user].append(f13)
#     industry_times[user].append(f14)
    
    product_advertiser[user].append(f1)
    product_industry[user].append(f2)
    advertiser_industry[user].append(f3)
user_ids = pd.DataFrame({
                        'user_id':list(product_advertiser.keys()),
#                          'creative_id':list(creative_id.values()),
#                         'ad_id':list(ad_id.values()),
#                          'product_id':list(product_id.values()),
#                          'product_category':list(product_category.values()),
#                          'advertiser_id':list(advertiser_id.values()),
#                          'industry':list(industry.values()),
#                          'click_times':list(click_times.values()),
#                          'time':list(time.values()),
                         
#                          'creative_id_times':list(creative_id_times.values()),
#                          'ad_id_times':list(ad_id_times.values()),
#                          'product_id_times':list(product_id_times.values()),
#                          'advertiser_id_times':list(advertiser_id_times.values()),
#                          'product_category_times':list(product_category_times.values()),
#                          'industry_times':list(industry_times.values()),
                        
                        'product_advertiser':list(product_advertiser.values()),
                        'product_industry':list(product_industry.values()),
                         'advertiser_industry':list(advertiser_industry.values()),
                        })
print(user_ids.shape)
user_ids.head()

(4000000, 4)


,user_id,product_advertiser,product_industry,advertiser_industry
0,1,"[21694, 659, 22341, 22020, 3072, 35397, 22127,...","[13521, 241, 13435, 13521, 1744, 14184, 13414,...","[9295, 12579, 40680, 19942, 15203, 20015, 2362..."
1,2,"[355, 322, 552, 573, 3440, 355, 42357, 2873, 3...","[140, 123, 218, 218, 1860, 140, 13943, 1602, 1...","[31515, 14049, 25548, 35890, 1017, 31515, 4766..."
2,3,"[78708, 83993, 4300, 78409, 4290, 33224, 32879...","[45863, 50516, 2328, 45592, 2328, 13902, 13949...","[45483, 12898, 25460, 19060, 23370, 11683, 108..."
3,4,"[5091, 32710, 7825, 81491, 84399, 80398, 82805...","[2606, 13921, 3815, 48351, 50876, 47366, 49513...","[26721, 10285, 17503, 16185, 32629, 1073, 1332..."
4,5,"[21818, 24260, 355, 662, 43556, 637, 22139, 41...","[13516, 13570, 140, 241, 14180, 241, 13507, 14...","[15834, 1345, 31515, 12586, 53557, 3791, 24472..."


In [11]:
path_list = "./feature_series/"
for name in ['product_advertiser', 'product_industry', 'advertiser_industry']:
    np.save(f"{path_list}{name}_list_int.npy", user_ids[name])
    user_ids[name] = user_ids[name].map(lambda x: [str(i) for i in x])
    np.save(f"{path_list}{name}_list.npy", user_ids[name])